In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [3]:
#май
df_05 = pd.read_csv('data/2021-05-01_low.csv.zip', parse_dates=['ts'])

In [5]:
#Реионы Москвы
region = gpd.read_file('data/mo-shape/mo.shp')
region.rename(columns={'NAME': "Субъект"}, inplace = True)

In [6]:
#продолжительность
df_05_1 = pd.read_csv('data/duraion_2021-05-01_low.csv.zip', parse_dates=['ts'])

### Список точек с координатами и регионом

In [7]:
ap_uniq = df_05.drop_duplicates(subset=['ap_mac'])
ap_uniq['location'] = ap_uniq['gps'].map(lambda x: [x for x in str(x).strip('(').strip(')').split(",")])
ap_uniq['lat'] = (ap_uniq['location'].str[0]).astype(float)
ap_uniq['lon'] = (ap_uniq['location'].str[1]).astype(float)
ap_uniq['geometry'] = ap_uniq.location.apply(lambda x: Point(float(x[1]), float(x[0])))
ap_uniq.drop(['ts', 'device_id', 'user_id', 'gps', 'location'], axis=1, inplace = True)
ap_uniq.head(2)

,ap_mac,lat,lon,geometry
0,3420E31780D0,55.721146,37.606163,POINT (37.60616302490234 55.72114562988281)
1,8CFE74212CD0,55.737167,37.623783,POINT (37.62378311157227 55.73716735839844)


In [8]:
ap_point = gpd.GeoDataFrame({"ap_mac" : ap_uniq.ap_mac, "lat" : ap_uniq.lat, "lon" : ap_uniq.lon,
                             "geometry": ap_uniq.geometry})
ap_point.head(2)

,ap_mac,lat,lon,geometry
0,3420E31780D0,55.721146,37.606163,POINT (37.60616 55.72115)
1,8CFE74212CD0,55.737167,37.623783,POINT (37.62378 55.73717)


In [9]:
#left_join "within" 
merging = gpd.sjoin(ap_point, region, how="left", op="within")
merging.head(2)

,ap_mac,lat,lon,geometry,index_right,Субъект,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO
0,3420E31780D0,55.721146,37.606163,POINT (37.60616 55.72115),87,Якиманка,45286596,45384000,Центральный,45286000,ЦАО,Муниципальный округ
1,8CFE74212CD0,55.737167,37.623783,POINT (37.62378 55.73717),87,Якиманка,45286596,45384000,Центральный,45286000,ЦАО,Муниципальный округ


In [ ]:
#сохранение геопандас
#сначала содать папку resions
#merging[['ap_point', 'geometry', 'NAME', 'NAME_AO']].to_file("resions/resions.shp", encoding='utf-8')
#regions = gpd.read_file('resions/resions.shp')

In [10]:
#сохранение для анализа
merging[['ap_mac', 'lat', 'lon', 'Субъект']].to_csv('data/uniq_point.csv', index=False)

### Статистики по точкам

Группировка по часовым инервалам

In [11]:
df_05['day'] = df_05['ts'].dt.day  #Понедельник 0
df_05['day_week'] = df_05['ts'].dt.dayofweek
df_05['hour'] = df_05['ts'].dt.hour
df_05['count']=1
df_05.head(2)

,ts,ap_mac,device_id,user_id,gps,day,day_week,hour,count
0,2021-05-26 09:24:05+00:00,3420E31780D0,655994ae339fcdbd0886c3a3fe29b73a,NaN,"(55.72114562988281, 37.606163024902344)",26,2,9,1
1,2021-05-26 09:24:05+00:00,8CFE74212CD0,18d69a401b87499778fb99a67bcb3155,NaN,"(55.73716735839844, 37.623783111572266)",26,2,9,1


In [12]:
df_05_hour = df_05[['ap_mac', 'day', 'day_week', 'hour', 'count']].groupby(['ap_mac', 'day', 'day_week','hour']).agg('count').reset_index()
df_05_hour.head(2)

,ap_mac,day,day_week,hour,count
0,0CF4D52B8E70,1,5,1,19
1,0CF4D52B8E70,1,5,2,8


In [13]:
#среднее в час
hour_count = pd.pivot_table(df_05_hour, values='count', index=['ap_mac'], columns=['hour'], aggfunc='mean')#
hour_count.columns = ['hour_' + str(i) for i in range(0, 24)] 
hour_count.reset_index(inplace=True)
hour_count.head(2)

,ap_mac,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0CF4D52B8E70,10.444444,7.375000,10.653846,16.310345,31.483871,34.935484,50.612903,51.709677,46.096774,...,66.322581,67.870968,73.580645,66.258065,57.838710,41.161290,25.774194,17.516129,9.400000,10.000000
1,0CF4D52B8EA0,11.214286,7.928571,12.666667,17.741935,54.451613,123.806452,127.967742,128.129032,133.387097,...,157.870968,174.161290,167.483871,134.161290,112.612903,81.935484,59.612903,33.032258,16.548387,12.724138


Группировка по дням

In [14]:
df_05_day = df_05[['ap_mac', 'day', 'day_week', 'count']].groupby(['ap_mac', 'day', 'day_week',]).agg('count').reset_index()
df_05_day.head(2)

,ap_mac,day,day_week,count
0,0CF4D52B8E70,1,5,866
1,0CF4D52B8E70,2,6,721


In [15]:
#среднее в день недели 1 - понедельник, 7 -воскресенье
week_count = pd.pivot_table(df_05_day, values='count', index=['ap_mac'], columns=['day_week'], aggfunc='mean')
week_count.columns = ['day_' + str(i) for i in range(1, 8)] 
week_count.reset_index(inplace=True)
week_count.head(2)

,ap_mac,day_1,day_2,day_3,day_4,day_5,day_6,day_7
0,0CF4D52B8E70,902.6,962.75,988.00,982.50,1104.0,967.4,919.6
1,0CF4D52B8EA0,2374.8,2420.00,2540.75,2463.75,2605.5,2064.4,1866.0


In [16]:
#группировка по пользователям
df_05_user = df_05.groupby(['ap_mac']).agg(device_count=('device_id', 'count'),
                                           device_nunique = ('device_id', 'nunique'),
                                           ).reset_index()
df_05_user['device_frec']=df_05_user['device_count']/df_05_user['device_nunique']
df_05_user.head(2)

,ap_mac,device_count,device_nunique,device_frec
0,0CF4D52B8E70,30097,5910,5.092555
1,0CF4D52B8EA0,71646,13194,5.430196


In [17]:
#группировка по продолжительности
def q1(x):
    return x.quantile(0.05)

def q2(x):
    return x.quantile(0.95)

df_05_dur = df_05_1.groupby(['ap_mac']).agg(user_nunique=('user_id', 'nunique'),
                                            duration = ('duration', 'median'),
                                            duration_min = ('duration', q1),
                                            duration_max = ('duration', q2),
                                           ).reset_index()
df_05_dur.head(2)

,ap_mac,user_nunique,duration,duration_min,duration_max
0,0CF4D52B8E70,97,37.0,3.95,185.2
1,0CF4D52B8EA0,185,27.0,4.00,132.0


Объединение статистики по трафику

In [18]:
df_points = pd.merge(hour_count, week_count, how='left', on=['ap_mac']) 
df_points = pd.merge(df_points, df_05_user, how='left', on=['ap_mac']) 
df_points = pd.merge(df_points, df_05_dur, how='left', on=['ap_mac']) 
df_points.head(2)

,ap_mac,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,day_5,day_6,day_7,device_count,device_nunique,device_frec,user_nunique,duration,duration_min,duration_max
0,0CF4D52B8E70,10.444444,7.375000,10.653846,16.310345,31.483871,34.935484,50.612903,51.709677,46.096774,...,1104.0,967.4,919.6,30097,5910,5.092555,97.0,37.0,3.95,185.2
1,0CF4D52B8EA0,11.214286,7.928571,12.666667,17.741935,54.451613,123.806452,127.967742,128.129032,133.387097,...,2605.5,2064.4,1866.0,71646,13194,5.430196,185.0,27.0,4.00,132.0


In [19]:
df_points.to_csv('data/df_points.csv', index=False)